# Importing the dataset

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
import cv2

#keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D,AveragePooling2D
from keras.utils import np_utils
import sklearn.metrics as metrics

train_data = pd.read_csv("../raw_data/kaggled/written_name_train_v2.csv", delimiter = ',')

test_data = pd.read_csv("../raw_data/kaggled/written_name_test_v2.csv", delimiter = ',')

#mapp = pd.read_csv("../raw_data/dataemnist/emnist-balanced-mapping.txt", delimiter = ' ',index_col=0, header=None, squeeze=True)





In [13]:
print("Train: %s, Test_data: %s" %(train_data.shape, test_data.shape))

Train: (330961, 2), Test_data: (41370, 2)


In [14]:
train_data.head()


,FILENAME,IDENTITY
0,TRAIN_00001.jpg,BALTHAZAR
1,TRAIN_00002.jpg,SIMON
2,TRAIN_00003.jpg,BENES
3,TRAIN_00004.jpg,LA LOVE
4,TRAIN_00005.jpg,DAPHNE


## Separating Features and Target

In [4]:
Height = 28
Width = 28


In [5]:
X_train = train_data.iloc[:,1:]
y_train = train_data.iloc[:,0]
del train_data

X_test = test_data.iloc[:,1:]
y_test = test_data.iloc[:,0]
del test_data



In [6]:
#X_train = train_data.iloc[:,1:]
#y_train = train_data.iloc[:,0]

In [7]:
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

(330961, 1) (330961,) (41370, 1) (41370,)


In [8]:
#X_test = test_data.iloc[:,1:]
#y_test = test_data.iloc[:,0]

In [9]:
#del train_data
#del test_data

## Rotate the immage

In [10]:
def rotate(image):
    image = image.reshape([28, 28])
    image = np.fliplr(image)
    image = np.rot90(image)
    return image

In [11]:
X_train = np.asarray(X_train)
X_train = np.apply_along_axis(rotate, 1, X_train)
print ("X_train:",X_train.shape)

ValueError: cannot reshape array of size 1 into shape (28,28)

In [ ]:
X_test = np.asarray(X_test)
X_test = np.apply_along_axis(rotate, 1, X_test)
print ("X_test:",X_test.shape)

## Plotting some examples

In [ ]:
for i in range(100, 109):
    plt.subplot(330 + (i+1))
    plt.imshow(X_train[i], cmap = 'gray')
    #plt.title(chr(mapp[y_train[i]]))

In [20]:
i = 128;
img_dir = "../raw_data/kaggled"+train_data.loc[i, 'written_name_train_v2.csv']
image = cv2.imread(img_dir, cv2.IMREAD_GRAYSCALE)
new_img = cv2.resize(image, (128, 64))
plt.imshow(new_img, cmap = 'gray')
plt.show()

KeyError: 'written_name_train_v2.csv'

## Normalize

In [ ]:
X_train = X_train.astype('float32')
X_train /= 255
X_test = X_test.astype('float32')
X_test /= 255

## Number of classes

In [ ]:
num_classes = y_train.nunique()
num_classes

## Expanding Dimensions

In [ ]:
#from tensorflow.keras.backend import expand_dims
#import tensorflow as tf

#X_train = tf.keras.backend.expand_dims(X_train, axis=-1)
#X_test = tf.keras.backend.expand_dims(X_test, axis=-1)

## One hot encoding

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

y_train_cat = np_utils.to_categorical(y_train, 47)
y_test_cat = np_utils.to_categorical(y_test, 47)
print("y_train: ", y_train_cat.shape)
print("y_test: ", y_test_cat.shape)



## Reshaping

In [ ]:
#X_train = X_train.reshape([-1,H,W,1])
#X_test = X_test.reshape([-1,H,W,1])

In [ ]:
X_train = X_train.reshape(-1, Height, Width, 1)
X_test = X_test.reshape(-1, Height, Width, 1)

## plotting

In [ ]:
#for i in range(100, 109):
    #plt.subplot(330 + (i+1))
    #plt.imshow(X_train[i], cmap = 'gray')
    #plt.title(chr(mapp[y_train[i]]))

In [ ]:
#for i in range(100, 109):
    #plt.subplot(330 + (i+1))
    #plt.imshow(X_train[i], cmap=plt.get_cmap('gray'))
    #plt.title(chr(mapp[y_train[i]]))

## Expanding the data

In [ ]:
#from tensorflow.keras.backend import expand_dims
#import tensorflow as tf
#X_train = tf.keras.backend.expand_dims(X_train, axis=-1)
#X_test = tf.keras.backend.expand_dims(X_test, axis=-1)

In [ ]:
#y_train = np_utils.to_categorical(y_train, num_classes)
#y_test = np_utils.to_categorical(y_test, num_classes)
#print("train_y: ", y_train.shape)
#print("test_y: ", y_test.shape)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train_cat, y_test_cat = train_test_split(X_train, y_train_cat, test_size= 0.2, random_state=5)

## Building a model

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.callbacks import EarlyStopping

def initialize_model(input_shape):
    input_x = tf.keras.layers.Input(input_shape)
    X = tf.keras.layers.Conv2D(32,(5,5),padding='same',activation='relu')(input_x)
    X = tf.keras.layers.MaxPooling2D((2,2),strides=(2,2))(X)
    X = tf.keras.layers.Conv2D(64,(3,3),padding='same',activation='relu')(X)
    X = tf.keras.layers.MaxPooling2D((2,2),strides=(2,2))(X)
    X = tf.keras.layers.Conv2D(128,(3,3),padding='same',activation='relu')(X)
    X = tf.keras.layers.MaxPooling2D((2,2),strides=(2,2))(X)
    X = tf.keras.layers.Flatten()(X)
    X = tf.keras.layers.Dense(256,activation='relu')(X)
    X = tf.keras.layers.Dense(128,activation='relu')(X)
    X = tf.keras.layers.Dense(num_classes,activation='softmax')(X)
    model = tf.keras.Model(inputs=input_x, outputs=X)
    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
model = initialize_model((Height,Width,1))

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta = 0, patience=4, verbose = 1, mode = 'auto')


history = model.fit(X_train, y_train_cat,
                    epochs=10, 
                    batch_size=64,
                    callbacks = [es],
                    validation_data=(X_test,y_test_cat))

In [ ]:
print(model.evaluate(X_test, y_test_cat, verbose=1))


## Plotting accuracy and loss

In [ ]:
# Plot accuracy, validation accuracy

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Plot loss, validation loss

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model.save("../raw_data/neural_model")

In [ ]:
model.predict(X_train)